In [ ]:
###Required packages

## P-Aminostyrene Synthesis Model

In [ ]:
###Helper functions

###Model definition

###Objective function

In [ ]:
###Run sample optimization

####FIGURE: Sample results

In [ ]:
###Explore chemical robustness 

####FIGURE: Chemical robustness